<img src="./Assets/1.1 AgWorkHeaderImage.png">

## Tool Use

It combines MEMORY, REASONING, PLANNING, and __ACTIONS (often via EXTERNAL TOOLS)__ to fulfil TASKS.

Tools are methods that help LLMs accomplish actions that require interaction with entities outside of the model.

<img src="./Assets/2.1 Tool Use Single.png">

---

#### Load all Keys etc.

In [76]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## Simple Tool Calling using a CrewAI Agent

<img src="./Assets/2.3 Crewai logo.png"><img src="./Assets/2.2 CrewAI.png">

In [212]:
from crewai import LLM
llm = LLM(
    model="gpt-4o-mini",
    temperature=0.8,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)

### Agent: Role, Goal & BackStory

In [213]:
from crewai import Agent
from crewai_tools import SerperDevTool

Query_Responder=Agent(
    role="AI Web Search Assistant",
    goal=f"Help users by either answering their queries directly using built-in knowledge or performing a live web search using" 
    "the Google Serper API when up-to-date or external information is required.",
    backstory=f"You were built to bridge the gap between real-time information and artificial intelligence. "
                "Equipped with access to the web through the Serper API, you excel at retrieving the most recent, relevant data when needed. " 
                "But you also know when to keep things simple — using your own understanding to answer straightforward questions without delay."  
                "You’re the perfect blend of real-time access and inherent intelligence.",
    tools=[SerperDevTool()],
    verbose=True,
    llm=llm
)


In [214]:
from crewai import Task

handle_user_query= Task(
    description=(
        "Receive and respond to a user's question - {user_input} "
        "If the query is simple or general knowledge (e.g., definitions, common facts), "
        "answer using your own knowledge. "
        "If the query appears to require current or web-based information "
        "(e.g., news, trending topics, recent data), "
        "perform a web search using the Serper API tool to gather relevant information first. "
        "Ensure your response is concise, helpful, and clearly indicates whether it was based on a search or internal understanding."
    ),
    expected_output=(
        "A final answer to the user's question, either using internal knowledge or based on live search results. "
        "If using search, summarize the result clearly and include key information retrieved. "
        "Indicate whether the result was from the web or internal knowledge."
        "Always end your answer by saying which source you used: [LLM] or [Web Search]"
    ),
    agent=Query_Responder
)

In [215]:
from crewai import Crew, Process

crew = Crew(
    agents=[Query_Responder],
    tasks=[handle_user_query],
    process=Process.sequential
)


In [216]:
result = crew.kickoff(inputs={"user_input": "Who won the latest t20 world cup?"})

# Agent: AI Web Search Assistant
## Task: Receive and respond to a user's question - Who won the latest t20 world cup? If the query is simple or general knowledge (e.g., definitions, common facts), answer using your own knowledge. If the query appears to require current or web-based information (e.g., news, trending topics, recent data), perform a web search using the Serper API tool to gather relevant information first. Ensure your response is concise, helpful, and clearly indicates whether it was based on a search or internal understanding.


# Agent: AI Web Search Assistant
## Thought: Action: Search the internet with Serper
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"latest T20 World Cup winner 2024\"}"
## Tool Output: 
{'searchParameters': {'q': 'latest T20 World Cup winner 2024', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'IND vs SA Cricket Scorecard, Final at Bridgetown, June 29, 2024', 'link': 'https://www.es

In [217]:
from IPython.display import Markdown, display
display(Markdown("### "+result.raw))

### India won the latest T20 World Cup in 2024, defeating South Africa by seven runs in the final held on June 29, 2024. India scored 176 runs while South Africa managed 169 runs. Virat Kohli was named Player of the Match for his innings of 76 runs off 59 balls. This victory marked India's first T20 World Cup title since 2014 and they became the first team to win the tournament undefeated. [Web Search]

In [219]:
result.token_usage

UsageMetrics(total_tokens=6273, prompt_tokens=6137, cached_prompt_tokens=0, completion_tokens=136, successful_requests=2)

---

## AtraDB Vector Search

In [179]:
Token=os.environ["ASTRA_TOKEN"]
EndPoint=os.environ["ASTRA_ENDPOINT"]

In [180]:
from astrapy import DataAPIClient

# Initialize the client
client = DataAPIClient(Token)
db = client.get_database_by_api_endpoint(
  EndPoint
)

print(f"Connected to Astra DB: {db.list_collection_names()}")

Connected to Astra DB: ['CollectionSmallEmbeddings']


In [181]:
collection = db.get_collection("CollectionSmallEmbeddings")

In [182]:
query="How many days of paternity leave can be availed?"

In [183]:
from openai import OpenAI
client = OpenAI()

query_embedding=client.embeddings.create(
  model="text-embedding-3-small",
  input=query,
  encoding_format="float"
)


In [195]:
from astrapy.data_types import DataAPIVector

results = collection.find(
    sort={"$vector": DataAPIVector(query_embedding.data[0].embedding)},
    limit=5,
    include_similarity=True
)


In [197]:
for doc in results:
    print(doc)


{'_id': '693147c55d554642bab1ed63f2285c55', 'page_content': 'childbirth.\n \nEligible\n \nemployees\n \nare\n \nentitled\n \nto\n \nup\n \nto\n \n7\n \ndays\n \nof\n \npaid\n \npaternity\n \nleave,\n \nwhich\n \ncan\n \nbe\n \nused\n \nwithin\n \n6\n \nmonths\n \nof\n \nthe\n \nchild’s\n \nbirth.\n ○  Employees  are  required  to  provide  proof  of  the  birth  (e.g.,  a  birth  certificate  or  \nhospital\n \nrecords)\n \nwhen\n \napplying\n \nfor\n \npaternity\n \nleave.\n \n \nf.  Long-Term  Leave  for  Medical  Conditions  or  Personal  Circumstances  \n1.  Leave  for  Long-Term  Medical  Conditions:  \n ○  Employees  dealing  with  long-term  medical  conditions  that  require  extended  time  \noff\n \nmay\n \nrequest\n \nlong-term\n \nmedical\n \nleave\n.\n \nThis\n \nleave\n \nis\n \ntypically\n \nunpaid\n \nand\n \nrequires\n \nmedical\n \ndocumentation\n \nto\n \nsubstantiate\n \nthe\n \nrequest.\n ○  The  company  will  review  each  case  individually  and  may  approve  l

---

## AstraDB Vector Tool (Custom CrewAI Tools)

In [160]:
from astrapy import DataAPIClient
from astrapy.data_types import DataAPIVector
from openai import OpenAI

def search_documents(query: str, top_k: int = 5) -> list[str]:
    # Set up OpenAI
    openai_client = OpenAI()
    query_embedding = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=query,
        encoding_format="float"
    ).data[0].embedding

    # Connect to Astra
    astra_client = DataAPIClient(Token)
    db = astra_client.get_database_by_api_endpoint(EndPoint)
    collection = db.get_collection("CollectionSmallEmbeddings")

    # Query Astra
    cursor = collection.find(
        sort={"$vector": DataAPIVector(query_embedding)},
        limit=top_k,
        include_similarity=True
    )

    # Return documents
    results = []
    for doc in cursor:
        text = doc.get("text") or str(doc)
        score = doc.get("$similarity", 0)
        results.append(f"[{score:.4f}] {text}")
    return results


In [161]:
from crewai.tools import BaseTool

class AstraSearchTool(BaseTool):
    name: str = "VectorTool"
    description: str = "Search the vector store for information about employee leave policy of AKAIWorks"

    def _run(self, query: str) -> str:
        # This is where your vector search logic goes
        results = search_documents(query)
        return "\n".join(results)


In [162]:
astra_tool = AstraSearchTool()

---

## Tool Routing using CrewAI

<img src="./Assets/2.4 Tool Routing.png">

In [220]:
llm = LLM(
    model="gpt-4o-mini",
    temperature=0.8,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)


In [221]:
Query_Responder=Agent(
    role="AI Search Assistant",
    goal="Help users by either answering their queries directly using built-in knowledge,"
        " or use the VectorTool when the query is about employee leave policy of AKAIWorks "
        "or performing a live web search using the Google Serper API when up-to-date or external information is required.",

    backstory="You were built to bridge the gap between real-time information and artificial intelligence. "
            "Equipped with access to the web through the Serper API, employee leave policy via the vector tool, you excel at retrieving the most recent, relevant data when needed."
            " But you also know when to keep things simple — using your own understanding to answer straightforward questions without delay."
            " You’re the perfect blend of real-time access and inherent intelligence. "
            "Remember that company name might not be explicitly mentioned but assume that the leave policy query is implicitly for the organisation.""",
    tools=[SerperDevTool(),astra_tool],
    verbose=True,
    llm=llm
)


In [222]:
handle_user_query= Task(
    description=(
        "Receive and respond to a user's question - {user_input} "
        "If the query is simple or general knowledge (e.g., definitions, common facts), "
        "answer using your own knowledge. "
        "If the query appears to require current or web-based information "
        "(e.g., news, trending topics, recent data), "
        "perform a web search using the Serper API tool to gather relevant information first. "
        "If the query is about the employee leave policy of AKAIWorks, "
        "perform a vector database search using vector search tool."
        "Remember that company name might not be explicitly mentioned but assume that the leave policy query is implicitly for the organisation."
        "Ensure your response is concise, helpful, and clearly indicates whether it was based on a search or internal understanding."
        "Always end your answer by saying which source you used: [LLM], [Web Search], or [Vector DB]."
    ),
    expected_output=(
        "A final answer to the user's question, either using internal knowledge or based on live search results. "
        "If using search, summarize the result clearly and include key information retrieved. "
        "Indicate whether the result was from the web or internal knowledge."
        "Always end your answer by saying which source you used: [LLM], [Web Search], or [Vector DB]."
    ),
    agent=Query_Responder
)

In [223]:
crew = Crew(
    agents=[Query_Responder],
    tasks=[handle_user_query],
    process=Process.sequential
)


In [224]:
result = crew.kickoff(inputs={"user_input": "How many paternity leaves are employees entitled to?"})

# Agent: AI Search Assistant
## Task: Receive and respond to a user's question - How many paternity leaves are employees entitled to? If the query is simple or general knowledge (e.g., definitions, common facts), answer using your own knowledge. If the query appears to require current or web-based information (e.g., news, trending topics, recent data), perform a web search using the Serper API tool to gather relevant information first. If the query is about the employee leave policy of AKAIWorks, perform a vector database search using vector search tool.Remember that company name might not be explicitly mentioned but assume that the leave policy query is implicitly for the organisation.Ensure your response is concise, helpful, and clearly indicates whether it was based on a search or internal understanding.Always end your answer by saying which source you used: [LLM], [Web Search], or [Vector DB].




# Agent: AI Search Assistant
## Thought: Action: VectorTool
## Using tool: VectorTool
## Tool Input: 
"{\"query\": \"paternity leave policy AKAIWorks\"}"
## Tool Output: 
[0.8236] {'_id': 'c0dedc882dc742bf8355a4ec4501516f', 'page_content': 'the\n \nMaternity\n \nBenefit\n \nAct,\n \n1961\n,\n \nand\n \nother\n \napplicable\n \nlaws\n \nin\n \nIndia.\n \nFemale\n \nemployees\n \nare\n \nentitled\n \nto\n \nup\n \nto\n \n26\n \nweeks\n \nof\n \npaid\n \nmaternity\n \nleave,\n \nprovided\n \nthey\n \nmeet\n \nthe\n \neligibility\n \ncriteria.\n ○  The  company  recognizes  that  maternity  leave  is  an  important  right  and  \nencourages\n \nemployees\n \nto\n \ninform\n \nHR\n \nas\n \nsoon\n \nas\n \npossible\n \nto\n \nmake\n \narrangements\n \nfor\n \nleave.\n ○  Employees  must  provide  medical  documentation  confirming  pregnancy  and  \nexpected\n \ndelivery\n \ndates,\n \nand\n \nthey\n \nmust\n \napply\n \nfor\n \nmaternity\n \nleave\n \nat\n \nleast\n \n8\n \nweeks\n \nbef

In [225]:
display(Markdown("### "+result.raw))


### Employees at AKAIWorks are entitled to up to 7 days of paid paternity leave, which can be utilized within 6 months of the child’s birth. This leave is provided to male employees in the case of childbirth. [Vector DB]

---

So we saw how to call external applications using __Tools__, now we will move to [Agents solving multiple tasks in steps](./3.%20MultiStep%20Agents.ipynb)

---
---

<img src="./Assets/profile_s.png" width=100> 

Hi! I'm Abhinav! I have spent over 15 years consulting and leadership roles in data science, machine learning and AI. My current focus is in the applied Generative AI domain focussing on solving enterprise needs through contextual intelligence. I'm passionate about AI advancements constantly exploring emerging technologies to push the boundaries and create positive impacts in the world. Let’s build the future, together!

[If you haven't already, get your copy of A Simple Guide to Retrieval Augmented Generation here](https://mng.bz/6ePo)

<a href="https://mng.bz/6ePo" target="_blank">
    <img src="./Assets/NewMEAPFooter.png" alt="New MEAP" width= 70%" />
</a>

#### If you'd like to chat, I'd be very happy to connect

[![GitHub followers](https://img.shields.io/badge/Github-000000?style=for-the-badge&logo=github&logoColor=black&color=orange)](https://github.com/abhinav-kimothi)
[![LinkedIn](https://img.shields.io/badge/LinkedIn-000000?style=for-the-badge&logo=linkedin&logoColor=orange&color=black)](https://www.linkedin.com/comm/mynetwork/discovery-see-all?usecase=PEOPLE_FOLLOWS&followMember=abhinav-kimothi)
[![Medium](https://img.shields.io/badge/Medium-000000?style=for-the-badge&logo=medium&logoColor=black&color=orange)](https://medium.com/@abhinavkimothi)
[![Insta](https://img.shields.io/badge/Instagram-000000?style=for-the-badge&logo=instagram&logoColor=orange&color=black)](https://www.instagram.com/akaiworks/)
[![Mail](https://img.shields.io/badge/email-000000?style=for-the-badge&logo=gmail&logoColor=black&color=orange)](mailto:abhinav.kimothi.ds@gmail.com)
[![X](https://img.shields.io/badge/Follow-000000?style=for-the-badge&logo=X&logoColor=orange&color=black)](https://twitter.com/abhinav_kimothi)
[![Linktree](https://img.shields.io/badge/Linktree-000000?style=for-the-badge&logo=linktree&logoColor=black&color=orange)](https://linktr.ee/abhinavkimothi)
[![Gumroad](https://img.shields.io/badge/Gumroad-000000?style=for-the-badge&logo=gumroad&logoColor=orange&color=black)](https://abhinavkimothi.gumroad.com/)

---